<a href="https://colab.research.google.com/github/nallagondu/DATASCIENCE-practice/blob/main/GICC_INC_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from wordcloud import WordCloud, STOPWORDS
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from google.colab import drive
from matplotlib import pyplot as plt


In [ ]:

drive.mount('/content/drive')

In [ ]:
# Load the data from Google Drive
data_path = '/content/drive/MyDrive/Colab Notebooks/DATA for Colab/incident52000.csv'
df = pd.read_csv(data_path)
df.head()

In [ ]:
df.tail()


In [ ]:
df.shape


In [ ]:
df.describe()

In [ ]:
missing_values = df.isnull().sum()
missing_values

In [ ]:
df.drop_duplicates()

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(columns=['Activity due','Work start','Timeline','Additional assignee list', 'Approval set', 'Due Date', 'SLA due'], inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
mode_assigned_to = df['Assigned to'].mode()[0]  # Calculate mode (most frequent value)
df['Assigned to'].fillna(mode_assigned_to, inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
# Function to extract server name
def extract_server_name(description):
    parts = description.split('_-_')
    server_name = parts[-1].strip()
    server_name = server_name.replace('[Server]', '').strip()
    return server_name

# Apply the function to extract server names only for rows where 'Causal Configuration Item' is empty
mask = df['Causal Configuration Item'].isnull()
df.loc[mask, 'Causal Configuration Item'] = df.loc[mask, 'Short Description'].apply(extract_server_name)

# Print the DataFrame with the extracted server names filled into the 'Causal Configuration Item' column
print(df)

In [ ]:
df

In [ ]:
df.isnull().sum()

In [ ]:
# Fill empty values in 'Resolved' with corresponding values from 'Created'
df['Resolved'].fillna(df['Created'], inplace=True)
df

In [ ]:
df.isnull().sum()

In [ ]:
df.head()


In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# Extract server name from 'Short Description' column
df['Server'] = df['Causal Configuration Item'].str.split('_-_').str[-1].str.strip()

# Encode categorical variables
label_encoder = LabelEncoder()
df['Server'] = label_encoder.fit_transform(df['Server'])
# Save the DataFrame to cleaned.csv
df.to_csv('cleanedData.csv', index=False)
df

In [ ]:
df['Created by'] = label_encoder.fit_transform(df['Created by'])
df['Severity'] = label_encoder.fit_transform(df['Severity'])

df.head()

In [ ]:
# Fill missing values in categorical columns with the most frequent value
categorical_cols = ['Caller', 'Causal Configuration Item', 'Description', 'Priority', 'Assigned to', 'State', 'Assignment group', 'Server']
for col in categorical_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

In [ ]:
# Fill missing values in numerical columns with median or mean
numerical_cols = ['Business duration', 'Child Incidents']
for col in numerical_cols:
    df[col].fillna(df[col].median(), inplace=True)
df.head()

In [ ]:
# Convert categorical variables into numerical representations using label encoding
#label_encoder = LabelEncoder()
#for col in categorical_cols:
  #  df[col] = label_encoder.fit_transform(df[col])

# Encode categorical columns: Caller, Server
label_encoder = LabelEncoder()
df['Caller'] = label_encoder.fit_transform(df['Caller'])
df

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def preprocess_text(text):
    # Check if the input is a list
    if isinstance(text, list):
        # Join list elements into a single string
        text = ' '.join(text)
    # Convert to lowercase
    text = text.lower()
    # Implement any additional preprocessing steps here
    return text

In [ ]:
# Apply text preprocessing
df['Short Description'] = df['Short Description'].apply(preprocess_text)
df['Description'] = df['Description'].apply(preprocess_text)

In [ ]:
# Vectorization using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # Adjust max_features as needed
X_short_description = tfidf_vectorizer.fit_transform(df['Short Description']).toarray()
X_description = tfidf_vectorizer.fit_transform(df['Description']).toarray()

# Indexing
short_description_index = {i: vector for i, vector in enumerate(X_short_description)}
description_index = {i: vector for i, vector in enumerate(X_description)}

# Function to perform retrieval
def retrieve_similar_documents(query, index, vectorizer, dataset):
    query_vector = vectorizer.transform([preprocess_text(query)]).toarray()

    # Calculate cosine similarity between query vector and vectors in the index
    similarity_scores = {}
    for i, vector in index.items():
        similarity = cosine_similarity(query_vector, [vector])[0][0]
        similarity_scores[i] = similarity

    # Retrieve top similar documents
    top_k = 10  # Number of top similar documents to retrieve
    top_documents = sorted(similarity_scores, key=similarity_scores.get, reverse=True)[:top_k]
    top_documents_info = [(dataset.iloc[i]['Short Description'], dataset.iloc[i]['Description']) for i in top_documents]

    return top_documents_info

# Example of retrieval
query = "Ping Down"  # Replace with your query

print("Top similar documents based on Short Description:")
top_short_description = retrieve_similar_documents(query, short_description_index, tfidf_vectorizer, df)
for short_desc, desc in top_short_description:
    print(short_desc)

print("\nTop similar documents based on Description:")
top_description = retrieve_similar_documents(query, description_index, tfidf_vectorizer, df)
for short_desc, desc in top_description:
    print(desc)

In [ ]:
# Function to perform retrieval and count occurrences
def retrieve_and_count_occurrences(query, index, vectorizer, dataset):
    query_vector = vectorizer.transform([preprocess_text(query)]).toarray()

    # Calculate cosine similarity between query vector and vectors in the index
    similarity_scores = {}
    for i, vector in index.items():
        similarity = cosine_similarity(query_vector, [vector])[0][0]
        similarity_scores[i] = similarity

    # Retrieve top similar documents
    top_k = 10  # Number of top similar documents to retrieve
    top_documents = sorted(similarity_scores, key=similarity_scores.get, reverse=True)[:top_k]
    top_documents_info = [(dataset.iloc[i]['Short Description'], dataset.iloc[i]['Description']) for i in top_documents]

    # Count occurrences of query in the dataset
    occurrences_count = sum(1 for _, desc in top_documents_info if query.lower() in desc.lower())

    return occurrences_count, top_documents_info

# Example of retrieval and occurrences count
query = "AUSPWESUPCSDB01"  # Replace with your query

occurrences_count_short_description, top_short_description = retrieve_and_count_occurrences(query, short_description_index, tfidf_vectorizer, df)
print(f"Occurrences count based on Short Description: {occurrences_count_short_description}")
for short_desc, desc in top_short_description:
    print(short_desc)

occurrences_count_description, top_description = retrieve_and_count_occurrences(query, description_index, tfidf_vectorizer, df)
print(f"\nOccurrences count based on Description: {occurrences_count_description}")
for short_desc, desc in top_description:
    print(desc)


In [ ]:
# Function to perform retrieval and count occurrences
def retrieve_and_count_occurrences(query, index, vectorizer, dataset):
    query_vector = vectorizer.transform([preprocess_text(query)]).toarray()

    # Calculate cosine similarity between query vector and vectors in the index
    similarity_scores = {}
    for i, vector in index.items():
        similarity = cosine_similarity(query_vector, [vector])[0][0]
        similarity_scores[i] = similarity

    # Retrieve top similar documents
    top_k = 10  # Number of top similar documents to retrieve
    top_documents = sorted(similarity_scores, key=similarity_scores.get, reverse=True)[:top_k]
    top_documents_info = [(dataset.iloc[i]['Causal Configuration Item'], dataset.iloc[i]['Description']) for i in top_documents]

    # Count occurrences of query in the dataset
    occurrences_count = sum(1 for server, _ in top_documents_info if query.lower() in server.lower())

    # Analyze repeated issues
    issues_count = {}
    for _, desc in top_documents_info:
        if query.lower() in desc.lower():
            if desc in issues_count:
                issues_count[desc] += 1
            else:
                issues_count[desc] = 1

    # Find the most repeated issue
    most_repeated_issue = max(issues_count, key=issues_count.get)

    return occurrences_count, most_repeated_issue, top_documents_info

# Example of retrieval and occurrences count
query = "AUSPWESUPCSDB01"  # Replace with your query

occurrences_count_short_description, most_repeated_issue_short_description, top_short_description = retrieve_and_count_occurrences(query, short_description_index, tfidf_vectorizer, df)
print(f"Occurrences count based on Short Description: {occurrences_count_short_description}")
print(f"Most repeated issue based on Short Description: {most_repeated_issue_short_description}")
for server, desc in top_short_description:
    print(f"Server: {server}, Issue: {desc}")

occurrences_count_description, most_repeated_issue_description, top_description = retrieve_and_count_occurrences(query, description_index, tfidf_vectorizer, df)
print(f"\nOccurrences count based on Description: {occurrences_count_description}")
print(f"Most repeated issue based on Description: {most_repeated_issue_description}")
for server, desc in top_description:
    print(f"Server: {server}, Issue: {desc}")


In [ ]:
# Process Short Description column by splitting into chunks
# Define a function to split text into chunks
#def split_into_chunks(text, chunk_size=50):
 #   chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
 #   return chunks

In [ ]:
# Apply the function to Short Description column
#df['Short Description'] = df['Short Description'].apply(split_into_chunks)

# Check the updated DataFrame
#print(df.head())

In [ ]:
df.head()